In [ ]:
from math import log
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import ode
from scipy.integrate import odeint

# Problem 1

In [ ]:
r0 = 5
w0 = 3

a = 1.
alpha = .5
c = .75
gamma = .25

t_f = 20
y0 = [r0, w0]

def predator_prey(t, y, a, alpha, c, gamma):
    return [y[0]*(a-alpha*y[1]), y[1]*(-c+gamma*y[0])]

t = np.linspace(0, t_f, 5*t_f)
y = np.zeros((len(t), len(y0)))
y[0,:] = y0

predator_prey_ode = lambda t, y:predator_prey(t, y, a, alpha, c, gamma)
p_p_solver = ode(predator_prey_ode).set_integrator('dopri5')
p_p_solver.set_initial_value(y0, 0)

for j in range(1, len(t)):
    y[j,:] = p_p_solver.integrate((t[j]))

# y = y.T

plt.plot(t, y[:,0], label='rabbit')
plt.plot(t, y[:,1], label='wolf')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Population')
plt.show()
# plt.savefig('Predator_Prey.pdf')

# Preliminaries

In [ ]:
rho_F = 9400.    # Constants given before the problem
rho_L = 1800.
gamma_F = 3.2
gamma_L = 22.
eta_F = 180.
eta_L = 230.
C = 10.4 # Forbes constant
beta_AT = 0.14 # Adaptive Thermogenesis
beta_TEF = 0.1 # Thermic Effect of Feeding
K = 0

def forbes(F):
    C1 = C * rho_L / rho_F
    return C1 / (C1 + F)

def energy_balance(F, L, EI, PAL):
    p = forbes(F)
    a1 = (1. / PAL - beta_AT) * EI - K - gamma_F * F - gamma_L * L
    a2 = (1 - p) * eta_F / rho_F + p * eta_L / rho_L + 1. / PAL
    return a1 / a2

def weight_odesystem(t, y, EI, PAL):
    F, L = y[0], y[1]
    p, EB = forbes(F), energy_balance(F, L, EI, PAL)
    return np.array([(1 - p) * EB / rho_F , p * EB / rho_L])

def fat_mass(BW, age, H, sex):
    BMI = BW / H**2.
    if sex == 'male':
        return BW * (-103.91 + 37.31 * log(BMI) + 0.14 * age) / 100
    else:
        return BW * (-102.01 + 39.96 * log(BMI) + 0.14 * age) / 100

# def initialize_all(a,b,y0,h):
#     '''
#     Copied over from lab 1
#     '''
#     n = int((b-a)/h+1)
#     X = np.linspace(a,b,n)
#     if isinstance(y0,np.ndarray):
#         Y = np.empty((n, y0.size))
#     else:
#         Y = np.empty(n)
#     Y[0] = y0
#     return X,Y,h,int(n)

# def mod_RK4(f,X,Y,h,n,EI,PAL):
#     '''
#     Modified Runga-Kutta method from previous lab. The only change is this can accept parameters EI and PAL
#     '''
#     for i in xrange(n-1):
#         K1 = f(X[i],Y[i],EI,PAL)
#         K2 = f(X[i]+h/2,Y[i]+h/2*K1,EI,PAL)
#         K3 = f(X[i]+h/2,Y[i]+h/2*K2,EI,PAL)
#         K4 = f(X[i+1],Y[i]+h*K3,EI,PAL)
#         Y[i+1] = Y[i] + h/6*(K1+2*K2+2*K3+K4)
#     return Y

# Problem 2

In [ ]:
# First change weight and height to metric
BW = 72.5748
age = 38
height = 1.7272
sex = 'female'

# Set F0 and L0 using the fat_mass(function)
F0 = fat_mass(BW, age, height, sex)
L0 = BW - F0

# Our derivative is with respect to time (days) 
# so we want to run for 5*365 days with a step size of one day
x_f = 5*365
# h = 1.

# Set parameters. Two sets are given, but the first is extraneous. 
# We are concerned with the EI and PAL moving forward.
EI = 2025
PAL = 1.5

# Set up all initial conditions needed
X = np.linspace(0, x_f, x_f)
Y0 = np.array([F0, L0])
Y = np.zeros((len(X), len(Y0)))
Y[0,:] = Y0

weight_ode = lambda x, y: weight_odesystem(x, y, EI, PAL)
IVP = ode(weight_ode).set_integrator('dopri5')
IVP.set_initial_value(Y0, 0)

for j in range(1, len(X)):
    Y[j,:] = IVP.integrate((X[j]))



# X,Y,h,n = initialize_all(0,t,np.array([F0,L0]),h)
# Y = mod_RK4(weight_odesystem,X,Y,h,n,EI,PAL).T



Y = Y.transpose()*2.20462

plt.plot(X,Y[0],label='Fat')
plt.plot(X,Y[1],label='Lean')
plt.plot(X,Y[0]+Y[1],label='Total')
plt.xlabel('days')
plt.ylabel('Weight')
plt.legend(loc=0)
plt.show()

# Problem 3

In [ ]:
# Initial Conditions, remmeber to change height and weight to metric.
BW = 72.5748
age = 26
height = 1.7272
sex = 'female'

F0 = fat_mass(BW, age, height, sex)
L0 = BW - F0

# Time period is 7 days * 16 weeks for each part of the weightloss prevention
x_f = 16*7
# h=1.

# Initial Ei and PAL - again ignore the extraneous conditions of what was done before the weightloss prevention started
EI = 1600
PAL = 1.7

# Run over the first t days
# X1,Y,h,n = initialize_all(0,t,np.array([F0,L0]),h)
# Y1 = mod_RK4(weight_odesystem,X1,Y,h,n,EI,PAL)

X1 = np.linspace(0, x_f, x_f)
Y0 = np.array([F0, L0])
Y1 = np.zeros((len(X1), len(Y0)))
Y1[0,:] = Y0

weight_ode = lambda x, y: weight_odesystem(x, y, EI, PAL)
IVP = ode(weight_ode).set_integrator('dopri5')
IVP.set_initial_value(Y0, 0)

for j in range(1, len(X1)):
    Y1[j,:] = IVP.integrate((X1[j]))

# Update the initial conditions
F0, L0 = Y1[-1,0], Y1[-1,1]
EI = 2025
PAL = 1.5

# Run over the next t days
# X2,Y,h,n = initialize_all(t,2*t,np.array([F0,L0]),h)
# Y2 = mod_RK4(weight_odesystem,X2,Y,h,n,EI,PAL)

X2 = np.linspace(x_f, 2*x_f, x_f)
Y0 = np.array([F0, L0])
Y2 = np.zeros((len(X2), len(Y0)))
Y2[0,:] = Y0

weight_ode = lambda x, y: weight_odesystem(x, y, EI, PAL)
IVP = ode(weight_ode).set_integrator('dopri5')
IVP.set_initial_value(Y0, 0)

for j in range(1, len(X2)):
    Y2[j,:] = IVP.integrate((X2[j]))

# Combine X and Y values
X = np.concatenate((X1,X2))
Y = np.concatenate((Y1,Y2)).T*2.20462

print (Y.shape)

plt.plot(X,Y[0],label='Fat')
plt.plot(X,Y[1],label='Lean')
plt.plot(X,Y[0]+Y[1],label='Total')
plt.xlabel('Time (days)')
plt.ylabel('Weight (lbs)')
plt.legend(loc=0)
plt.show()

# Problem 4

In [ ]:
# Initial Conditions
a, b = 0., 13.
alpha = 1./3
dim = 2
y0 = np.array([3./4,3./4])

# Differential Equation where x is time, y is 2 dimensional. This is designed to solve derivatives at each point simultaneously.
def Lotka_Volterra(y,x):
    return np.array([y[0]*(1.-y[1]), alpha*y[1]*(y[0]-1.)])

subintervals = 200

# Create grid to find derivatives
Y1, Y2 = np.meshgrid(np.arange(0, 4.5, .2), np.arange(0, 4.5, .2), sparse=True, copy=False)

# Find derivative at each point
U, V = Lotka_Volterra((Y1, Y2), 0)
Q = plt.quiver(Y1[::3,::3], Y2[::3,::3], U[::3,::3], V[::3,::3], pivot='mid', color='b', units='dots', width=3.)

# Plot fixed points
plt.plot(1, 1, 'ok', markersize=8)
plt.plot(0, 0, 'ok', markersize=8)

# Plot specific solutions
inits = [np.array([3./4,3./4]),np.array([1./2,3./4]),np.array([1./16,3./4]),np.array([1./40,3./4])]
for conditions in inits:
    Y = odeint(Lotka_Volterra, conditions, np.linspace(a, b, subintervals))
    plt.plot(Y[:,0], Y[:,1], '-k', linewidth=2.0)
    plt.plot(Y[::10,0], Y[::10,1], '*r')

plt.axis([-.5, 4.5, -.5, 4.5])
plt.title("Phase Portrait of the Lotka-Volterra Predator-Prey Model")
plt.xlabel('Prey',fontsize=15)
plt.ylabel('Predators',fontsize=15)
plt.show()

# Problem 5

In [ ]:

"""Using scipy.integrate.odeint, approximate the predator-prey trajectories.
Plot and show the solutions. Create two plots, one for each pair of 
parameter values alpha, beta. 
"""
a, b = 0., 13.          # (Nondimensional) Time interval for one 'period'
dim = 2                 # dimension of the system
subintervals = 200

def single_solve(alpha=1, beta=.3):
    """For a given alpha and beta, plot two solutions."""
    def Logistic(y, x):
        return np.array([y[0] * (1.-y[0]-y[1]), alpha*y[1]*(y[0]-beta)])

    def solve_Log(y0):    
        """Use the built in ode solver."""
        return odeint(Logistic, y0, np.linspace(a, b, subintervals))

    # Plot the direction field
    Y1, Y2 = np.meshgrid(np.arange(0, 4.5, .2), np.arange(0, 4.5, .2),
                                                sparse=True, copy=False)
    U, V = Logistic((Y1, Y2), 0)
    Q = plt.quiver(Y1[::3, ::3], Y2[::3, ::3], U[::3, ::3], V[::3, ::3],
                            pivot='mid', color='b', units='dots',width=3.)

    # You must find new equilibrium points
    plt.plot(1, 0, 'ok', markersize=8)
    plt.plot(0, 0, 'ok', markersize=8)
    plt.plot(beta, 1-beta, 'ok', markersize=8)

    def plot_solution(Y):
        """Plot the solution in phase space"""
        plt.plot(Y[:,0], Y[:,1], '-k', linewidth=2.0)
        plt.plot(Y[::10,0], Y[::10,1], '*b')

    plot_solution(solve_Log(np.array([1 / 3., 1 / 3.])))
    plot_solution(solve_Log(np.array([1 / 2., 1 / 5.])))

plt.subplot(121)
single_solve(1.,.3)
plt.xlabel('Prey',fontsize=15)
plt.ylabel('Predators',fontsize=15)
plt.axis([-.5, 4.5, -.5, 4.5])
plt.subplot(122)
single_solve(1.,1.1)
plt.xlabel('Prey',fontsize=15)
plt.ylabel('Predators',fontsize=15)
plt.axis([-.5, 4.5, -.5, 4.5])
plt.suptitle("Phase Portrait of the Logistic Predator-Prey Model")

plt.show()